# TensorFlow入門1 基本的な仕組み

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
tf.__version__

'1.14.0'

## 1.このテキストについて

### テキストの目的
- TensorFlowの基本的な仕組みを知る

### どのように学ぶか
サンプルコードとともに説明していきます。同じコードを打ち込んで実行していってください。

## 2.TensorFlowとは

TensorFlowはニューラルネットワークを構築するためのフレームワークの一種です。

以下のコードはバージョン1.13.0での動作を確認しています。

### 簡単な計算

簡単な計算をさせてみます。TensorFlowの基本的な流れです。

**《サンプルコード》**

In [3]:
import tensorflow as tf
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)
sess = tf.Session()
output = sess.run(add)
print(output) # 12
sess.close()

12


7行目の`sess = tf.Session()`から`sess.close()`は`with`を使って以下のように書くこともできます。

In [4]:
with tf.Session() as sess:
    output = sess.run(add)
    print(output) # 12

12


比較のために、NumPyでも同じ計算をさせてみます。

In [5]:
import numpy as np
a_n = np.array(5)
b_n = np.array(7)
output_n = np.add(a_n, b_n)
print(output_n) # 12

12


このような簡単な計算例ではNumPyの方が短く記述可能です。TensorFlowの真価が発揮されるのは学習を行うような計算の際ですが、ここでは簡単な計算を例に理解を深めていくことにします。

## 3.データフロープログラミング

TensorFlowの仕組みを見ていきます。TensorFlowでは`データフローグラフ`を考えます。

簡単な計算の例では以下のようなデータフローグラフになります。

![image](https://t.gyazo.com/teams/diveintocode/b10659507cc836291a26834ace057152.png)

グラフはノード（丸い部分）とエッジ（線の部分）で表されます。ノードはオペレーション（操作）を表し、エッジはTensor（多次元配列）を表します。

今回の例では左側の灰色のノードでは定数を作るオペレーション（操作）、右の水色のノードでは足し算のオペレーション（操作）が行われています。エッジは整数値です。

### TensorFlowの手順
TensorFlowの手順は以下のようになっています。

1. データフローグラフを構築する
2. データを入力して計算する

簡単な計算の例でそれぞれを解説します。

### データフローグラフの構築
サンプルコードの以下の部分でデータフローグラフを構築しています。

In [6]:
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)

定数aと定数bを定義し、それらを足し算するデータフローグラフです。NumPyであれば3行で計算まで行いますが、TensorFlowでは実際の計算までは進みません。

それぞれをprintしてみても、エッジ（Tensor）の説明が返されるだけです。

In [7]:
print(a) # Tensor("Const:0", shape=(), dtype=int32)
print(add) # Tensor("Add:0", shape=(), dtype=int32)

Tensor("Const_2:0", shape=(), dtype=int32)
Tensor("Add_1:0", shape=(), dtype=int32)


### データを入力して計算
データフローグラフの計算を行う際には`セッション`という概念が登場します。

まずSessionオブジェクトを作成します。

In [8]:
sess = tf.Session()

そして、`sess.run()`の中にエッジ（Tensor）を入れます。そうすることで、そのエッジ（Tensor）がどのような値を持つかが返ってきます。

In [9]:
output = sess.run(add)
print(output) # 12

12


データフローグラフやセッションの概念は、次のテキストでより複雑な計算を実装していくことで理解を深めます。

### TensorFlowにおける値の扱い方
TensorFlowが値を扱う上で独自の概念として、placeholderとValiableがあります。

#### 《placeholder》

placeholderはデータフローグラフの構築時には値が決まっていないものに使います。最初は配列の形だけ定義しておいて後から値を入れて使う空箱のような存在です。学習ごとに違う値が入る入力データや正解データなどに用いられます。

[tf.placeholder  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/compat/v1/placeholder?authuser=0&hl=ja)

サンプルコードをplaceholderを使用したものに書き換えてみます。

In [10]:
c = tf.placeholder(tf.int32)
d = tf.placeholder(tf.int32)
add = tf.add(c, d)
sess = tf.Session()
output = sess.run(add, feed_dict={c:5, d:7})
print(output) # 12

12


セッションを実行する際に引数`feed_dict`を使い、placeholderに入れる値を辞書型で指定します。ここを書き換えることで異なる計算が可能になります。

In [11]:
output = sess.run(add, feed_dict={c:20, d:32})
print(output) # 52

52


今回の簡単な例では定数とplaceholderの違いは感じられませんが、ミニバッチ学習を行うような場合を想定すると、必要性がわかります。

#### 《Valiable》

Valiableはplaceholderとは違い、データフローグラフの構築時にも値を持ち、更新を行うものに対して使います。学習するパラメータ（重み、バイアス）に用いられます。

[tf.Variable  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/Variable)

これについては次のテキストで詳しく見ていきます。

#### 《constant》

確認になりますが、placeholderでもValiableでもないただの値は定数constantとして扱います。

[tf.constant  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/constant)

### TensorFlowにおける演算

サンプルコードでは`tf.add()`を使用していますが、NumPyなどと同様に`+`を使用することも可能です。
```
add = a + b # tf.add(a, b)に等しい
```

TensorFlowではNumPyなどと同様に様々な関数が用意されています。

### セッションの終了
最後にセッションは終了させます。

In [12]:
sess.close()

セッションのインスタンス化から終了までに対して、with構文を使うことも可能です。
```
with tf.Session() as sess:
    sess.run() # ここに計算の実行コードを入れていく
```